<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Data-Augmentation/Analysis" data-toc-modified-id="Data-Augmentation/Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Augmentation/Analysis</a></span></li><li><span><a href="#Dataloader-creation-and-test" data-toc-modified-id="Dataloader-creation-and-test-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dataloader creation and test</a></span></li><li><span><a href="#Model-creation" data-toc-modified-id="Model-creation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model creation</a></span></li><li><span><a href="#Model-Training" data-toc-modified-id="Model-Training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model Training</a></span><ul class="toc-item"><li><span><a href="#Model-Testing" data-toc-modified-id="Model-Testing-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Model Testing</a></span></li></ul></li></ul></div>

**GET DATA**

Summary of this notebook: ...

Definition of Done: ...

# Imports


In [1]:
# Imports from Colab 2
import math
import numpy as np
import pickle
import keras
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import model architecture
# from tensorflow.keras.applications import VGG16

Using TensorFlow backend.


In [2]:
# Imports for Colab 6
import sys
#sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

import cv2 # Read raw image
import glob
# from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
# from scipy import ndimage # For rotation task or
# import imutils
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.python.keras.utils import data_utils
from tensorflow.keras.preprocessing.image import Iterator


# Imports for Colorizer
from os import path
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.layers import Reshape, ZeroPadding2D, Add
from tensorflow.keras.layers import Activation, InputLayer, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# from skimage.color import rgb2lab, lab2rgb, rgb2gray
# from skimage.io import imsave
import random
import tensorflow as tf
from skimage.transform import resize
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import RMSprop
import time

import tensorflow.keras.backend as K

In [3]:
# Check to see if GPU is being used
tensorflow.test.gpu_device_name()


'/device:GPU:0'

# Dataloader creation and test

In [4]:
# dims=(448,448,3)

# train_name='../ssd/proxy_data/merced_xtrain.npy'
# test_name='../ssd/proxy_data/merced_xtest.npy'

# if path.exists(train_name):
#   print("Train exists")
#   X_train=np.load(train_name)
# else:
#   X_train=np.empty((0,*dims))

# if path.exists(test_name):
#   print("Test exists")
#   X_test=np.load(test_name)
# else:
#   X_test=np.empty((0,*dims))

# X_train_reshaped=np.zeros((len(X_train),448,448,3))
# X_test_reshaped=np.zeros((len(X_test),448,448,3))

# for i in range(len(X_train)):
#     X_train_reshaped[i,:]=resize(X_train[i],dims)
# print("Done")
# del X_train
# for i in range(len(X_test)):
#     X_test_reshaped[i,:]=resize(X_test[i],dims)
# print("Done")
# del X_test

# np.save('../ssd/proxy_data/merced_xtrain_448.npy',X_train_reshaped)
# np.save('../ssd/proxy_data/merced_xtest_448.npy',X_test_reshaped)


dims=(448,448,3)

train_name='../ssd/proxy_data/merced_xtrain_448.npy'
test_name='../ssd/proxy_data/merced_xtest_448.npy'

if path.exists(train_name):
  print("Train exists")
  X_train_reshaped=np.load(train_name)
else:
  X_train=np.empty((0,*dims))

if path.exists(test_name):
  print("Test exists")
  X_test_reshaped=np.load(test_name)
else:
  X_test=np.empty((0,*dims))

Train exists
Test exists


In [5]:
batch_size = 32

AUTOTUNE=tensorflow.data.experimental.AUTOTUNE

def convert(image, label):
    image = tensorflow.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
    label = tensorflow.image.convert_image_dtype(label, tf.float32)
    return image, label


# def parse_data(feat,label):
#     feat=Resizing(448,448,interpolation='bicubic')
#     label=Resizing(448,448,interpolation='bicubic')
#     return feat,label


train_dataset = tf.data.Dataset.from_tensor_slices((X_train_reshaped, X_train_reshaped))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_reshaped, X_test_reshaped))

train_dataset = train_dataset.map(convert, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.cache().shuffle(buffer_size=12*batch_size)
train_dataset = train_dataset.batch(batch_size)
# train_dataset = train_dataset.repeat()
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

test_dataset = test_dataset.map(convert, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.cache()
test_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
def batch_lab(batch_size,data_generator,data): # Does basically nothing, but just to help with later tasks
  for batch in data_generator.flow(data,batch_size=batch_size):
    batch=resize(batch,(batch_size,*dims))
#     print(np.max(batch),np.min(batch))
    yield batch,batch

# Model creation

In [6]:
def encoder(input_shape):

    model = Sequential(name="encoder")
    model.add(Input(shape=input_shape))
    model.add(Conv2D(32, (3, 3), padding="same", strides=2))
    model.add(PReLU())
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2D(64, (3, 3), padding="same", strides=2))
    model.add(PReLU())
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2D(128, (3, 3), padding="same", strides=2))
    model.add(PReLU())
    model.add(Conv2D(256, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2D(256, (3, 3), padding="same", strides=2))
    model.add(PReLU())
    model.add(Conv2D(15, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2D(5, (3, 3), padding="same"))
    model.add(PReLU())
    return model

def decoder(input_shape):
    model = Sequential(name="decoder")
    model.add(Input(shape=input_shape))
    model.add(Conv2DTranspose(128, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(64, (3, 3),padding="same"))
    model.add(PReLU())
    model.add(Conv2DTranspose(64, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(32, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(Conv2D(3, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(3, (3, 3), padding="same"))
    model.add(PReLU())
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(3, (3, 3), activation="tanh", padding="same"))
    return model

In [ ]:
encoder_model=encoder(dims)
encoding_depth=15

decoder_model=decoder(encoder_model.output_shape[1:])

complete_model=Sequential()
complete_model.add(encoder_model)
complete_model.add(decoder_model)

complete_model.build(input_shape=(None,*dims))
print(complete_model.summary())

# Model Training

In [ ]:
complete_model.compile(optimizer='rmsprop', loss='mse')
complete_model.summary()

In [ ]:
complete_model.fit(train_dataset,
                    epochs=100,
                    steps_per_epoch=len(X_train_reshaped)/batch_size,
                    validation_data=test_dataset,
                    validation_steps=len(X_test_reshaped)/batch_size)


In [ ]:
complete_model.save('../ssd/proxy_models/ae_epoch100_ucmerced')

## Model Testing

In [ ]:
model=load_model('../ssd/proxy_models/ae_epoch100_ucmerced')

In [ ]:

for i in range(10):
    index=np.random.randint(0,len(X_test_reshaped))

    X_test_im=np.expand_dims(X_test_reshaped[index],0)
    out_image=np.squeeze(model.predict(X_test_im))
    
    im_min=out_image.min(axis=(0, 1), keepdims=True)
    im_max=out_image.max(axis=(0, 1), keepdims=True)
    out_image=(out_image-im_min)/(im_max-im_min)
    
    
    print("Orig ",np.min(X_test_im),np.max(X_test_im))
    print("Gen ",np.min(out_image),np.max(out_image))
    fig=plt.figure()
    plt.subplot(1, 3, 1)
    plt.imshow(X_test_reshaped[index])
    plt.subplot(1,3,2)
    plt.imshow(np.squeeze(X_test_im))
    plt.subplot(1,3,3)
    plt.imshow(out_image)
    plt.show()

## Modifying Loss Function

In [7]:
encoder_model=encoder(dims)
encoding_depth=15

decoder_model=decoder(encoder_model.output_shape[1:])

model=Sequential()
model.add(encoder_model)
model.add(decoder_model)

model.build(input_shape=(None,*dims))

# model.compile(optimizer='rmsprop', loss=custom_loss)
# model.summary()
opt=RMSprop(learning_rate=1e-3)

In [8]:
def custom_loss(y_true,y_pred): # Issue-> Non differentiable function
    # y_true,y_pred of the shape (bs,h,w,n_c)
    loss=tf.Variable(0.,tf.float32)
    for i in range(3):
        y_true_ch=tf.reshape(y_true[:,:,:,i],[-1])
        y_pred_ch=tf.reshape(y_pred[:,:,:,i],[-1])
        y_true_hist=tf.histogram_fixed_width(y_true_ch, value_range=(0., 1.), nbins=10) 
        y_pred_hist=tf.histogram_fixed_width(y_pred_ch, value_range=(0., 1.), nbins=10) 
        y_true_hist=tf.cast(y_true_hist,dtype=tf.dtypes.float32)
        y_pred_hist=tf.cast(y_pred_hist,dtype=tf.dtypes.float32)
        ch_loss=K.mean(K.square(y_true_hist-y_pred_hist))
#         print(loss)
        loss=tf.math.add(loss,ch_loss)
#     print(loss)
    return loss


def bce_loss(y_true,y_pred):
    loss=tf.Variable(0.,tf.float32)
    for i in range(3):
        bce = tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.25)
        y_true_ch=tf.reshape(y_true[:,:,:,i],[-1])
        y_pred_ch=tf.reshape(y_pred[:,:,:,i],[-1])
        loss_ch=bce(y_true_ch,y_pred_ch)
        loss=tf.math.add(loss,loss_ch)
    return loss
        
        
# @tf.function
def step(model,X,Y):
#     print(model.trainable_variables)
    with tf.GradientTape(watch_accessed_variables=True) as tape:
#         
        pred=model(X,training=True)
#         loss=custom_loss(Y,pred)
        loss=bce_loss(Y,pred)
#         tape.watch(model.trainable_variables)
    grads=tape.gradient(loss,model.trainable_variables)
    return loss,grads

# @tf.function
def validate(X,Y):
    pred=model(X,training=False)
    valid_loss=bce_loss(Y,pred)
    return valid_loss

In [9]:
train_loss=[]
val_loss=[]
for epoch in range(50):
    t_start=time.time()
    train_epoch_loss=tensorflow.keras.metrics.Mean()
    val_epoch_loss=tensorflow.keras.metrics.Mean()
    
    for batch_idx,(x,y) in enumerate(train_dataset):
        loss,grad=step(model,x,y)
#         print(grad)
        opt.apply_gradients(zip(grad,model.trainable_variables))
        
        
        train_epoch_loss.update_state(loss)
        if batch_idx%20==0:
            print("Train Batch Loss at Epoch %d step %d: %.4f"
                % (epoch,batch_idx, float(loss)))
    train_loss.append(train_epoch_loss.result())
                  
    for x_val,y_val in test_dataset:
        v_loss=validate(x_val,y_val)
        val_epoch_loss.update_state(v_loss)
    print("Train loss at Epoch %d: %.4f Time taken %.3f "%(epoch,train_epoch_loss.result(),time.time()-t_start))
    print("\t Val loss at Epoch %d: %.4f "%(epoch,val_epoch_loss.result()))
    val_loss.append(val_epoch_loss.result())
      


Train Batch Loss at Epoch 0 step 0: 15.6532
Train Batch Loss at Epoch 0 step 20: 2.7414
Train Batch Loss at Epoch 0 step 40: 2.1329
Train loss at Epoch 0: 3.2627 Time taken 75.892 
	 Val loss at Epoch 0: 3.6701 
Train Batch Loss at Epoch 1 step 0: 3.6251
Train Batch Loss at Epoch 1 step 20: 2.4028
Train Batch Loss at Epoch 1 step 40: 2.3118
Train loss at Epoch 1: 2.3610 Time taken 65.024 
	 Val loss at Epoch 1: 2.9484 
Train Batch Loss at Epoch 2 step 0: 2.9113
Train Batch Loss at Epoch 2 step 20: 2.2216
Train Batch Loss at Epoch 2 step 40: 2.2407
Train loss at Epoch 2: 2.2414 Time taken 64.921 
	 Val loss at Epoch 2: 2.0526 
Train Batch Loss at Epoch 3 step 0: 2.0435
Train Batch Loss at Epoch 3 step 20: 2.5713
Train Batch Loss at Epoch 3 step 40: 2.1471
Train loss at Epoch 3: 2.1793 Time taken 65.041 
	 Val loss at Epoch 3: 2.0732 
Train Batch Loss at Epoch 4 step 0: 2.0561
Train Batch Loss at Epoch 4 step 20: 2.0376
Train Batch Loss at Epoch 4 step 40: 2.0745
Train loss at Epoch 4: 2

KeyboardInterrupt: 

In [ ]:
model.save('../ssd/proxy_models/ae_epoch100_bceloss_ucmerced')

In [ ]:

for i in range(10):
    index=np.random.randint(0,len(X_test_reshaped))

    X_test_im=np.expand_dims(X_test_reshaped[index],0)
    out_image=np.squeeze(model.predict(X_test_im))
    
    im_min=out_image.min(axis=(0, 1), keepdims=True)
    im_max=out_image.max(axis=(0, 1), keepdims=True)
    out_image=(out_image-im_min)/(im_max-im_min)
    
    
    print("Orig ",np.min(X_test_im),np.max(X_test_im))
    print("Gen ",np.min(out_image),np.max(out_image))
    fig=plt.figure()
    plt.subplot(1, 3, 1)
    plt.imshow(X_test_reshaped[index])
    plt.subplot(1,3,2)
    plt.imshow(np.squeeze(X_test_im))
    plt.subplot(1,3,3)
    plt.imshow(out_image)
    plt.show()